In [ ]:
from ngsolve import *
from netgen.occ import *
#from netgen.occ import SplineApproximation
from ngsolve.webgui import Draw
from time import sleep

from netgen.webgui import Draw as DrawGeo

#widget stuff
import ipywidgets as widgets
from ipywidgets import interact, fixed

In [ ]:
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole
DrawGeo(shape)


In [ ]:
#mesh=Mesh(unit_square.GenerateMesh(maxh=0.1))
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1)).Curve(3)
Draw(mesh)


In [ ]:
fes = H1(mesh, order=3)
gf_sol = GridFunction(fes)
scene = Draw(gf_sol)

In [ ]:
fes_def=VectorH1(mesh,order=3,dirichlet=".*")

u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

gf_def_top=GridFunction(fes_def)
gf_def_bot=GridFunction(fes_def)
gf_def=GridFunction(fes_def)
gf_def_rad = GridFunction(fes_def)

delta_top=Parameter(-0.3)
delta_bot=Parameter(0.3)
delta_rad = Parameter(0.2)

#gf_def_top.Set((0, 4*x*(1-x)*y))
gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))
gf_def_bot.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("bottom"))
gf_def_rad.Set(5*CF((x-2.5, y-0.5)), definedon=mesh.Boundaries("circle"))


#gf_def_bot.Set((0, 4*x*(1-x)*(1-y)))
#gf_def.Set((0, delta_top*4*x*(1-x)*y + delta_bot*4*x*(1-x)*(1-y)))
#gf_def.vec.data = gf_def_top.vec + gf_def_bot.vec
# alternativ:
gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                  delta_bot.Get() * gf_def_bot.vec + \
                  delta_rad.Get() * gf_def_rad.vec
Draw(gf_def)

# Fun with Sliders

In [ ]:
fes = H1(mesh, order=3, dirichlet=".*")
gf_sol = GridFunction(fes)
#scene = Draw(gf_sol)

In [ ]:
def solve_problem(solution):
    fes = solution.space#H1(mesh, order=3, dirichlet=".*")
    
    u, v = fes.TnT()
    a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
    f = LinearForm(v*dx).Assemble()
    
    solution.vec[:] = a.mat.Inverse(freedofs=fes.FreeDofs())*f.vec

In [ ]:
mesh.SetDeformation(gf_def)
#mesh.SetDeformation(gf_def)

In [ ]:
V = VectorH1(mesh,order=3, dirichlet="inlet|top|bottom|circle")
Q = H1(mesh,order=2)
fes = V*Q

uin = CoefficientFunction( (1.5*4*y*(1.0-y), 0) )

gf_sol = GridFunction(fes)
gf_sol.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))


In [ ]:
def solve_problem(solution):
    fes = solution.space    
    
    u,p = fes.TrialFunction()
    v,q = fes.TestFunction()
    
    nu = 0.001  # viscosity
    stokes = (nu*InnerProduct(grad(u), grad(v))+ \
        div(u)*q+div(v)*p - 1e-10*p*q)*dx

    a = BilinearForm(stokes).Assemble()

    # nothing here ...
    f = LinearForm(fes).Assemble()
    
    inv_stokes = a.mat.Inverse(fes.FreeDofs())

    res = f.vec.CreateVector()
    res.data = f.vec - a.mat*solution.vec
    solution.vec.data += inv_stokes * res

In [ ]:
scene=Draw(gf_sol.components[0], mesh)
def change_grid_layout_3(delta_val_top, delta_val_bot, delta_val_rad):
    delta_rad.Set(delta_val_rad)
    # update upper deformation
    delta_top.Set(delta_val_top)
    # update lower deformation
    delta_bot.Set(delta_val_bot)
    # apply updated parameters to grid function
    # gf_def.Set((0, delta_top*4*x*(1-x)*y + delta_bot*4*x*(1-x)*(1-y)))
    gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                  delta_bot.Get() * gf_def_bot.vec + \
                  delta_rad.Get() * gf_def_rad.vec    
    gf_def.vec.data -= A_def_inv@a_def.mat*gf_def.vec # --> "how to set boundaries ngsolve tut"
    
    # appley grid function to mesh
    #gf_sol.Set(sin(20*delta_val_top*x))
    solve_problem(gf_sol)
    mesh.SetDeformation(gf_def)
    scene.Redraw()
    return;

interact(change_grid_layout_3, 
         delta_val_top=widgets.FloatSlider(
            value=0,
            min=-0.2,
            max=0.2,
            step=0.002,
            description='top:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.3f',
        ), 
        delta_val_bot=widgets.FloatSlider(
            value=0,
            min=-0.2,
            max=0.2,
            step=0.002,
            description='bottom:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.3f',
        ),
        delta_val_rad=widgets.FloatSlider(
            value=0,
            min=-0.1,
            max=0.1,
            step=0.002,
            description='radius:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.3f',
        ))

# Stokes H1 example:

In [ ]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
V.SetOrder(TRIG,3)
V.Update()
Q = L2(mesh, order=1)
X = V*Q

u,p = X.TrialFunction()
v,q = X.TestFunction()

a = BilinearForm(X)
a += (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a.Assemble()

gfu = GridFunction(X)
uin = CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

res = gfu.vec.CreateVector()
res.data = -a.mat * gfu.vec
inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="sparsecholesky")
gfu.vec.data += inv * res
Draw(gfu.components[0], mesh, "vel")
Draw(Norm(gfu.components[0]), mesh, "|vel|")
SetVisualization(max=2)

In [ ]:
V.SetOrder?

In [ ]:
help()